<a href="https://colab.research.google.com/github/fpd4fv/ds3002/blob/gh-pages/spacy_full_cycle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy==3.1.1
!pip install spacy-transformers
!python -m spacy download en_core_web_trf


     |████████████████████████████████| 6.4 MB 4.2 MB/s 
     |████████████████████████████████| 653 kB 43.5 MB/s 
     |████████████████████████████████| 10.1 MB 41.8 MB/s 
     |████████████████████████████████| 457 kB 48.3 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 58 kB 3.8 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: smart-open
    Found existing installation: smart-open 6.0.0
    Uninstalling smart-open-6.0.0:
      Successfully uninstall

In [ ]:
import spacy
import pandas as pd
from datetime import datetime
import spacy
import spacy_transformers

# Storing docs in binary format
from spacy.tokens import DocBin

In [ ]:
from zipfile import ZipFile

In [7]:
#Uploading model folder to colab
file_name = '/content/nlp_model_forzip.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')


Done


In [8]:
#Reading in trained model

#Test the data from the best model
#nlp = spacy.load("output_updated/model-best")
nlp = spacy.load("/content/nlp_model_forzip")

text = "Australia’s largest airline temporarily lays off 2,500 employees"
demo = nlp(text)
print(demo.cats)


{'positive': 0.0006150818080641329, 'negative': 0.9989302754402161, 'neutral': 0.00045462805428542197}


/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [9]:
#Bringing in Twitter API functionality
import tweepy
from tweepy import Stream
import datetime

In [10]:
#Twitter devel credentials
access_token="1433202678818017283-Tz047y4zcoa9ivP7r0UHlmuko2MqKd"
access_token_secret="OeubpAnY4u0yPp0Pckcor6589hNB9gU06dojxRGf2jw8A"
API_key="qfOhwhutCVNW80q0C4WnXBdPn"
API_secret_key="HAmpbaSuZNWgaAoLi7092QjsH2wS6jsXH991e6fjAJGc4KnIaI"

auth = tweepy.OAuthHandler(API_key, API_secret_key)
auth.set_access_token(access_token, access_token_secret)


In [11]:
api = tweepy.API(auth, wait_on_rate_limit=True)


In [21]:
### Getting user and printing their tweets
def user_tweets(twitter_handle, number_of_tweets_wanted):
  user_tweets = api.user_timeline(screen_name=twitter_handle , count = number_of_tweets_wanted)
  tweet_ids = [tweet.id for tweet in user_tweets]
  tweet_user = [twitter_handle]*len(user_tweets)
  tweet_dates = [tweet.created_at for tweet in user_tweets]
  tweet_texts = [tweet.text for tweet in user_tweets]
  #Producing df with these values
  user_tweets_dict = {
    'Tweet ID': tweet_ids,
    'Tweet Poster': tweet_user,
    'Tweet Date': tweet_dates,
    'Tweet Text': tweet_texts,
    }
  user_tweets_df = pd.DataFrame(user_tweets_dict)
  print(user_tweets_df.head())
  return user_tweets_df

  

In [60]:
#Function to see if tweet is talking about the US
relevant_to_US_words = ['US','U.S.','#US',"US'", 'United States','States',"States'",'America','Biden','NATO','#NATO',"#NATO's"]


In [61]:
### Getting user tweets about the U.S. from user. Assembling the tweets and their seniment analysis in a df
def user_tweets_about_US(twitter_handle):
  user_tweets = api.user_timeline(screen_name= twitter_handle, count=75, include_rts = False)
  relevant_tweets = []
  for tweet in user_tweets:
    tweet_words_list = tweet.text.split() #CONTINUE WORK HERE 01:46 5/04. IS THE SPLIT NOT PROCESSING HASHTAGS AND APOSTROPHES CORRECTLY???
    for word in tweet_words_list:
      if word in relevant_to_US_words:
        relevant_tweets.append(tweet)
  tweet_ids = []
  tweet_dates = []
  tweet_texts = []

  for tweet in relevant_tweets:
    tweet_ids.append(tweet.id)
    tweet_dates.append(tweet.created_at)
    tweet_texts.append(tweet.text)
  
  tweet_user = [twitter_handle]*len(relevant_tweets)

  #need to remove urls from text of tweets THIS NEEDS WORK

  ### conducting sentiment analysis on text of each tweet
  # initializing lists of each score type
  positive_scores = []
  negative_scores = []
  neutral_scores = []

  for tweet in relevant_tweets:
    demo = nlp(tweet.text)
    positive_scores.append(demo.cats['positive'])
    negative_scores.append(demo.cats['negative'])
    neutral_scores.append(demo.cats['neutral'])


  #Producing df with these values
  relevant_tweets_dict = {
    'Tweet ID': tweet_ids,
    'Tweet Poster': tweet_user,
    'Tweet Date': tweet_dates,
    'Tweet Text': tweet_texts,
    'Positive Score' : positive_scores,
    'Negative Score' : negative_scores,
    'Neutral Score' : neutral_scores
    }
  relevant_tweets_df = pd.DataFrame(relevant_tweets_dict)
  #print(relevant_tweets_df.head())
  return relevant_tweets_df

In [62]:
user_tweets_about_US('zlj517')

/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


,Tweet ID,Tweet Poster,Tweet Date,Tweet Text,Positive Score,Negative Score,Neutral Score
0,1521397793302380544,zlj517,2022-05-03 07:54:34,"Why the ""China threat""?\nBecause China exists ...",0.00051,0.000064,0.999427


In [40]:
tweet_text_1 = 'The US is the master of disinformation and coercive diplomacy, and the biggest saboteur of international and regional peace and stability. Island countries in the South Pacific are independent and sovereign states, not a backyard of the US or Australia. Reviving the Monroe Doctrine in the South Pacific region will get no support and lead nowhere.'
tweet_text_2 = 'US policy is clear. US leadership from Biden downwards has said aims are: Regime change, Proxy war, Unlimited resources for Ukraine, Prolonged war to weaken Russia, No peace negotiations.'
tweet_text_3 = "China's success in development is not built on establishing overseas military bases, interfering in others' internal affairs, imposing sanctions at will or plundering resources with guns and bombs."
tweet_text_4 = 'US is good at making wealth, especially from war and crisis.'
tweet_text_5 = "China strongly condemned the terrorist attack Tuesday afternoon inside a university in Pakistan's southern port city of Karachi, the Chinese Embassy in Pakistan said in a statement"
tweet_text_6 = 'In the first quarter, China imported RMB46.02 bn of agricultural products from ASEAN, a year-on-year increase of 14.1%.'


foreign_minister_tweets = [tweet_text_1 , tweet_text_2, tweet_text_3 , tweet_text_4, tweet_text_5 , tweet_text_6]